# Solving optimal control using Q-learning 
This notebook introduces the algorithm of deep Q-learning, for a discrete pendulum.

## Setup
For the notebook, we need tensorflow:

- pip install --user --upgrade pip

- pip install --user tensorflow==2.0.0

We will use a simple pendulum model with a single DOF. It is wrapped in the file tp5.env_pendulum. Three different versions are available: discrete state and discrete control in EnvPendulumDiscrete ; continuous state and discrete control in EnvPendulumHybrid ; and continuous state and control space in EnvPendulum.
Don't forget to run gepetto-viewer first is you want to display the model. The environments follows the API introduced in "Gym @ OpenIA": 
- env.reset() internally set the environment state.
- env.step() change the internal state and returns a pair (next_state,reward)
- env.render() display the internal state.

We also need a bunch of classic python stuff.

In [ ]:
import time
import signal
import matplotlib.pyplot as plt
import random
import numpy as np
import tensorflow as tf

## Q-Table
The first algorithm solves the discrete problem by representing the Q function Q(x,u) = l(x,u) + V(f(x,u)) as a table, and search the optimal Q function by enforcing the HJB equation. The policy is simply obtained as the argmax of the Q-table.

For this algorithm, we need the discrete pendulum:

In [ ]:
# %load -r 18-19 tp5/qtable.py
from tp5.env_pendulum import EnvPendulumDiscrete; Env = lambda : EnvPendulumDiscrete(1)
env = Env()

The hyperparameters are relatively few:

In [ ]:
# %load -r 22-25 tp5/qtable.py
NEPISODES               = 200           # Number of training episodes
NSTEPS                  = 50            # Max episode length
LEARNING_RATE           = 0.85          # 
DECAY_RATE              = 0.99          # Discount factor 

The Q function is represented by a table. Evaluating Q(x,u) is as simple as choosing the coefficient x,u in the table Q[x,u].

In [ ]:
# %load -r 27 tp5/qtable.py
Q     = np.zeros([env.nx,env.nu])       # Q-table initialized to 0

The algorithm runs a number of training episode NEPISODES. At each step of an episod, the Q table is adjusted to better match HJB.

In [ ]:
# %load -r 39-57 tp5/qtable.py
h_rwd = []                              # Learning history (for plot).
for episode in range(1,NEPISODES):
    x    = env.reset()
    rsum = 0.0
    for steps in range(NSTEPS):
        u         = np.argmax(Q[x,:] + np.random.randn(1,env.nu)/episode) # Greedy action with noise
        x2,reward = env.step(u)
        
        # Compute reference Q-value at state x respecting HJB
        Qref = reward + DECAY_RATE*np.max(Q[x2,:])

        # Update Q-Table to better fit HJB
        Q[x,u] += LEARNING_RATE*(Qref-Q[x,u])
        x       = x2
        rsum   += reward

    h_rwd.append(rsum)
    if not episode%20:
        print('Episode #%d done with average cost %.2f' % (episode,sum(h_rwd[-20:])/20))

You can try a roll-out with the following "rendertrial" method.

In [ ]:
# %load -r 29-35 tp5/qtable.py
def rendertrial(s0=None,maxiter=100):
    '''Roll-out from random state using greedy policy.'''
    s = env.reset(s0)
    for i in range(maxiter):
        a = np.argmax(Q[s,:])
        s,r = env.step(a)
        env.render()

In [ ]:
rendertrial()

## The Q-table algorithm as a neural algorithm
As an intermediate step before introducing the deep-Q algorithm, we suggest a modification of the Q-table algorithm by representing the Q-table as a neural network. To make it simple, the neural network is chosen with a trivial structure: a single layer, linear activation (i.e. it is a matrix!).
Then, the Q-table update becomes a gradient descent, which is less efficient but more generic and would scale adequately if we change the network structure.


In [2]:
# %load tp5/deeptable.py
'''
Example of Q-table learning with a simple discretized 1-pendulum environment using a linear Q network.
'''

import numpy as np
import random
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import matplotlib.pyplot as plt
from tp5.env_pendulum import EnvPendulumDiscrete; Env = lambda : EnvPendulumDiscrete(1)
import signal
import time
tf1.disable_eager_execution()


### --- Random seed
RANDOM_SEED = int((time.time()%10)*1000)
print("Seed = %d" % RANDOM_SEED)
np.random.seed(RANDOM_SEED)

### --- Hyper paramaters
NEPISODES               = 500           # Number of training episodes
NSTEPS                  = 50            # Max episode length
LEARNING_RATE           = 0.1           # Step length in optimizer
DECAY_RATE              = 0.99          # Discount factor 

### --- Environment
env = Env()
NX  = env.nx
NU  = env.nu

### --- Q-value networks
class QValueNetwork:
    def __init__(self):
        x               = tf1.placeholder(shape=[1,NX],dtype=tf.float32)
        W               = tf1.Variable(tf1.random_uniform([NX,NU],0,0.01,seed=100))
        qvalue          = tf1.matmul(x,W)
        u               = tf1.argmax(qvalue,1)

        qref            = tf1.placeholder(shape=[1,NU],dtype=tf.float32)
        loss            = tf1.reduce_sum(tf.square(qref - qvalue))
        optim           = tf1.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)

        self.x          = x             # Network input
        self.qvalue     = qvalue        # Q-value as a function of x
        self.u          = u             # Policy  as a function of x
        self.qref       = qref          # Reference Q-value at next step (to be set to l+Q o f)
        self.optim      = optim         # Optimizer      

### --- Tensor flow initialization
#tf.reset_default_graph()
qvalue  = QValueNetwork()
sess = tf1.InteractiveSession()
tf1.global_variables_initializer().run()

def onehot(ix,n=NX):
    '''Return a vector which is 0 everywhere except index <i> set to 1.'''
    return np.array([[ (i==ix) for i in range(n) ],],np.float)
   
def disturb(u,i):
    u += int(np.random.randn()*10/(i/50+10))
    return np.clip(u,0,NU-1)

def rendertrial(maxiter=100):
    x = env.reset()
    for i in range(maxiter):
        u = sess.run(qvalue.u,feed_dict={ qvalue.x:onehot(x) })
        x,r = env.step(u)
        env.render()
        if r==1: print('Reward!'); break
signal.signal(signal.SIGTSTP, lambda x,y:rendertrial()) # Roll-out when CTRL-Z is pressed

### --- History of search
h_rwd = []                              # Learning history (for plot).

### --- Training
for episode in range(1,NEPISODES):
    x    = env.reset()
    rsum = 0.0

    for step in range(NSTEPS-1):
        u = sess.run(qvalue.u,feed_dict={ qvalue.x: onehot(x) })[0] # Greedy policy ...
        u = disturb(u,episode)                                      # ... with noise
        x2,reward = env.step(u)

        # Compute reference Q-value at state x respecting HJB
        Q2        = sess.run(qvalue.qvalue,feed_dict={ qvalue.x: onehot(x2) })
        Qref      = sess.run(qvalue.qvalue,feed_dict={ qvalue.x: onehot(x ) })
        Qref[0,u] = reward + DECAY_RATE*np.max(Q2)

        # Update Q-table to better fit HJB
        sess.run(qvalue.optim,feed_dict={ qvalue.x    : onehot(x),
                                          qvalue.qref : Qref       })

        rsum += reward
        x = x2
        if reward == 1: break

    h_rwd.append(rsum)
    if not episode%20: print('Episode #%d done with %d sucess' % (episode,sum(h_rwd[-20:])))

print("Total rate of success: %.3f" % (sum(h_rwd)/NEPISODES))
rendertrial()
plt.plot( np.cumsum(h_rwd)/range(1,NEPISODES) )
plt.show()



Seed = 7943
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode #20 done with 0 sucess
Episode #40 done with 0 sucess
Episode #60 done with 0 sucess
Episode #80 done with 0 sucess
Episode #100 done with 0 sucess
Episode #120 done with 0 sucess
Episode #140 done with 0 sucess
Episode #160 done with 0 sucess
Episode #180 done with 0 sucess
Episode #200 done with 0 sucess
Episode #220 done with 0 sucess
Episode #240 done with 0 sucess
Episode #260 done with 0 sucess
Episode #280 done with 0 sucess
Episode #300 done with 0 sucess
Episode #320 done with 0 sucess
Episode #340 done with 0 sucess
Episode #360 done with 0 sucess
Episode #380 done with 0 sucess
Episode #400 done with 0 sucess
Episode #420 done with 0 sucess
Episode #440 done with 0 sucess
Episode #460 done with 0 sucess
Episode #480 done with 0 sucess
Total rate of success: 0.000


<Figure size 640x480 with 1 Axes>

## Deep-Q algorithm

The deep-Q algorithm is a subtil variation of the previous algorithm. The main difference is that ... it works. For that, we need a little bit of magic:
- we introduce a "target" network, which somehow smooth the excessive variations of the network weights introduced by the gradient descent.
- we keep the recent previous experiences in a memory buffer, that breaks the temporal structure when learning the Q function.

Now that we have a proper neural network, we can work with a continuous state. The first layers of the network will learn a proper representation of the state, no need to discretize it.


In [ ]:
from tp5.env_pendulum import EnvPendulumHybrid
env = EnvPendulumHybrid(1)

We have some more hyper parameters to tune the magic.

In [ ]:
# %load -r 32-40 tp5/qlearn.py
### --- Hyper paramaters
NEPISODES               = 1000          # Max training steps
NSTEPS                  = 60            # Max episode length
QVALUE_LEARNING_RATE    = 0.001         # Base learning rate for the Q-value Network
DECAY_RATE              = 0.99          # Discount factor 
UPDATE_RATE             = 0.01          # Homotopy rate to update the networks
REPLAY_SIZE             = 10000         # Size of replay buffer
BATCH_SIZE              = 64            # Number of points to be fed in stochastic gradient
NH1 = NH2               = 32            # Hidden layer size

We need a replay buffer to store the past experience. We store that in a "deque" where data can be added and removed from both side of the queue.

In [ ]:
from collections import deque

In [ ]:
# %load -r 42-51 tp5/qlearn.py
### --- Replay memory
class ReplayItem:
    def __init__(self,x,u,r,d,x2):
        self.x          = x
        self.u          = u
        self.reward     = r
        self.done       = d
        self.x2         = x2
replayDeque = deque()


To make the code more readable, the network itself is built in a dedicated class that we do not exhibit here.

In [ ]:
from tp5.qnetwork import QNetwork

In [ ]:
# %load -r 52-57 tp5/qlearn.py
### --- Tensor flow initialization
qvalue          = QNetwork(nx=env.nx,nu=env.nu,learning_rate=QVALUE_LEARNING_RATE)
qvalueTarget    = QNetwork(name='target',nx=env.nx,nu=env.nu)
# Uncomment to load networks
#qvalue.load()
#qvalueTarget.load()

We are now ready to run the episodes. At each step of each episode, we store the result of the experience in the memory. If there is enough material in the memory, we discard the oldest experiences. Then we run one step of gradient descent to adjust the Q-function to HJB equality.

In [ ]:
# %load -r 71-119 tp5/qlearn.py
### History of search
h_rwd = []

### --- Training
for episode in range(1,NEPISODES):
    x    = env.reset()
    rsum = 0.0

    for step in range(NSTEPS):
        u       = qvalue.policy(x,                                     # Greedy policy ...
                                noise=1. / (1. + episode + step))      # ... with noise
        x2,r    = env.step(u)
        done    = False # Some environment may return information when task completed

        replayDeque.append(ReplayItem(x,u,r,done,x2))                # Feed replay memory ...
        if len(replayDeque)>REPLAY_SIZE: replayDeque.popleft()       # ... with FIFO forgetting.

        rsum   += r
        x       = x2
        if done: break
        
        # Start optimizing networks when memory size > batch size.
        if len(replayDeque) > BATCH_SIZE:     
            batch = random.sample(replayDeque,BATCH_SIZE)            # Random batch from replay memory.
            x_batch    = np.vstack([ b.x      for b in batch ])
            u_batch    = np.vstack([ b.u      for b in batch ])
            r_batch    = np.array([ [b.reward] for b in batch ])
            d_batch    = np.array([ [b.done]   for b in batch ])
            x2_batch   = np.vstack([ b.x2     for b in batch ])
            
            # Compute Q(x,u) from target network
            v_batch    = qvalueTarget.value(x2_batch)
            qref_batch = r_batch + (d_batch==False)*(DECAY_RATE*v_batch)

            # Update qvalue to solve HJB constraint: q = r + q'
            qvalue.trainer.train_on_batch([x_batch,u_batch],qref_batch)
            
            # Update target networks by homotopy.
            qvalueTarget.targetAssign(qvalue,UPDATE_RATE)
      
    # \\\END_FOR step in range(NSTEPS)

    # Display and logging (not mandatory).
    print('Ep#{:3d}: lasted {:d} steps, reward={:3.0f}' .format(episode, step,rsum))
    h_rwd.append(rsum)
    if not (episode+1) % 200:     rendertrial(30)

# \\\END_FOR episode in range(NEPISODES)


As previously, we can run the policy with the following function:

In [ ]:
# %load -r 59-68 tp5/qlearn.py
def rendertrial(maxiter=NSTEPS,verbose=True):
    x = env.reset()
    rsum = 0.
    for i in range(maxiter):
        u = qvalue.policy(x)[0]
        x, reward = env.step(u)
        env.render()
        time.sleep(1e-2)
        rsum += reward
    if verbose: print('Lasted ',i,' timestep -- total reward:',rsum)

In [ ]:
rendertrial()